In [ ]:
%cd /workspace


In [ ]:
import os
print("cwd:", os.getcwd())
print("dirs:", os.listdir())


In [ ]:
import tensorflow as tf, numpy as np, pandas as pd
from tensorflow.keras import layers, models, callbacks, mixed_precision

# ---------- data ----------
train_ds = tf.keras.utils.image_dataset_from_directory(
    "ships32", validation_split=0.15, subset="training",
    seed=42, label_mode="int", image_size=(32, 32), batch_size=256)
val_ds = tf.keras.utils.image_dataset_from_directory(
    "ships32", validation_split=0.15, subset="validation",
    seed=42, label_mode="int", image_size=(32, 32), batch_size=256)

aug = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1)
])

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(lambda x, y: (aug(x) / 255., y), num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
val_ds   = val_ds.map(lambda x, y: (x / 255., y),     num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)


In [4]:


# ---------- model (<30 layers) ----------
mixed_precision.set_global_policy('mixed_float16')

def block(x, f):
    x = layers.Conv2D(f, 3, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)              # ← changement ici
    return x

inp = layers.Input((32, 32, 3))
x = block(inp, 32);  x = block(x, 32);  x = layers.MaxPool2D()(x)
x = block(x, 64);   x = block(x, 64);   x = layers.MaxPool2D()(x)
x = block(x, 128);  x = block(x, 128);  x = layers.MaxPool2D()(x)
x = block(x, 256)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.25)(x)
out = layers.Dense(13, activation='softmax', dtype='float32')(x)

model = models.Model(inp, out)
print("Nombre de couches :", len(model.layers))  # 28

model.compile(
    optimizer=tf.keras.optimizers.Adam(3e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

cbs = [
    callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(factor=0.5, patience=4, verbose=1)
]


model.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=cbs)




KeyboardInterrupt: 

In [1]:
# ---------- submission ----------
comp = np.load('ships_competition.npz')['X'] / 255.
pred = model.predict(comp, batch_size=512).argmax(1)
pd.DataFrame({'ID': np.arange(len(pred)), 'Category': pred}).to_csv('ships_competition.csv', index=False)


NameError: name 'np' is not defined

In [4]:
import tensorflow as tf, numpy as np, pandas as pd
import tensorflow_addons as tfa
from tensorflow.keras import layers, models, callbacks, mixed_precision

mixed_precision.set_global_policy('mixed_float16')

def se_block(x, ratio=8):
    filters = x.shape[-1]
    se = layers.GlobalAveragePooling2D()(x)
    se = layers.Dense(filters // ratio, activation='relu')(se)
    se = layers.Dense(filters, activation='sigmoid')(se)
    return layers.multiply([x, se])

def dsc_block(x, f):
    x = layers.SeparableConv2D(f, 3, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = se_block(x)
    return x

inp = layers.Input((32, 32, 3))
x = layers.Rescaling(1./255)(inp)
x = dsc_block(x, 32)
x = dsc_block(x, 32); x = layers.MaxPooling2D()(x)
x = dsc_block(x, 64); x = dsc_block(x, 64); x = layers.MaxPooling2D()(x)
x = dsc_block(x, 128); x = layers.MaxPooling2D()(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
out = layers.Dense(13, activation='softmax', dtype='float32')(x)

model = models.Model(inp, out)
print("Nombre de couches :", len(model.layers))  # ≈ 29

opt = tfa.optimizers.Lookahead(tfa.optimizers.RectifiedAdam(learning_rate=1e-3))

model.compile(
    optimizer=opt,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(label_smoothing=0.05),
    metrics=['accuracy']
)


2025-04-23 20:29:07.761730: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745432947.785514   22239 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745432947.792238   22239 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745432947.804555   22239 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745432947.804579   22239 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745432947.804581   22239 computation_placer.cc:177] computation placer alr

ModuleNotFoundError: No module named 'tensorflow_addons'

In [ ]:
import imgaug.augmenters as iaa
from tensorflow.keras.utils import Sequence
import os
from PIL import Image

class ShipDataset(Sequence):
    def __init__(self, root_dir, batch_size=256, img_size=32, augment=True):
        self.paths = []
        self.labels = []
        self.label_map = {name: i for i, name in enumerate(sorted(os.listdir(root_dir)))}
        for label in self.label_map:
            for file in os.listdir(os.path.join(root_dir, label)):
                self.paths.append(os.path.join(root_dir, label, file))
                self.labels.append(self.label_map[label])
        self.paths, self.labels = np.array(self.paths), np.array(self.labels)
        self.batch_size = batch_size
        self.img_size = img_size
        self.augment = augment
        self.aug = iaa.Sequential([
            iaa.Fliplr(0.5),
            iaa.Affine(rotate=(-15, 15)),
            iaa.AdditiveGaussianNoise(scale=0.01*255),
            iaa.GammaContrast((0.8, 1.2))
        ])

    def __len__(self):
        return len(self.paths) // self.batch_size

    def __getitem__(self, idx):
        batch_x = self.paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        images = np.stack([
            np.array(Image.open(p).resize((self.img_size, self.img_size))) for p in batch_x
        ])
        if self.augment:
            images = self.aug(images=images)
        return images, batch_y

train_gen = ShipDataset("ships32", batch_size=256, augment=True)
val_gen = ShipDataset("ships32", batch_size=256, augment=False)


In [ ]:
cbs = [
    callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(factor=0.5, patience=5, verbose=1)
]
model.fit(train_gen, validation_data=val_gen, epochs=100, callbacks=cbs)


NameError: name 'callbacks' is not defined

In [ ]:
comp = np.load('ships_competition.npz')['arr_0'] / 255.
pred = model.predict(comp, batch_size=512).argmax(1)
pd.DataFrame({'ID': np.arange(len(pred)), 'Category': pred}).to_csv('ships_competition.csv', index=False)
